In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType, TimestampType

import utils.data_processing_bronze_table

import utils.data_processing_silver_table
import utils.data_processing_gold_table


## set up pyspark session

In [2]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 04:37:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## set up config

In [3]:
# set up config
snapshot_date_str = "2023-01-01"
start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [4]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)
# dates_str_lst

## Build Bronze Table

In [5]:
data_directory = 'data'
list_of_raw_files = utils.data_processing_bronze_table.get_csv_files_in_directory(data_directory)
for file_name in list_of_raw_files:
    base_name = os.path.splitext(file_name)[0]
    directory_path = f"datamart/bronze/{base_name}/"
    csv_file_path = os.path.join(data_directory, file_name)
    utils.data_processing_bronze_table.process_bronze_datalake(
        directory_path=directory_path,
        dates_list=dates_str_lst,
        processing_function=utils.data_processing_bronze_table.process_bronze_table,
        spark=spark,
        csv_file_path=csv_file_path
    )

Created directory: datamart/bronze/lms_loan_daily/
Processing date: 2023-01-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-01-01
Processing date: 2023-02-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-02-01
Processing date: 2023-03-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-03-01
Processing date: 2023-04-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-04-01
Processing date: 2023-05-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-05-01
Processing date: 2023-06-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-06-01
Processing date: 2023-07-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-07-01
Processing date: 2023-08-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/date=2023-08-01
Processing date: 2023-09-01
Saved data to Parquet table at: datamart/bronze/lms_loan_daily/da

## Build Silver Table

In [6]:
processor = utils.data_processing_silver_table.DataProcessor(spark)
bronze_directory = 'datamart/bronze/feature_clickstream'
silver_directory = 'datamart/silver/feature_clickstream_cleaned'
processor.process_bronze_to_silver(
    bronze_path=bronze_directory,
    silver_path=silver_directory,
    dates=dates_str_lst,
    source_type="feature_clickstream"
)


2025-05-21 04:37:40,087 - INFO - Schema validation passed.
2025-05-21 04:37:40,382 - INFO - Processed DataFrame count for 2023-01-01: 8974
2025-05-21 04:37:40,385 - INFO - Processed DataFrame schema for 2023-01-01: StructType([StructField('fe_1', IntegerType(), True), StructField('fe_2', IntegerType(), True), StructField('fe_3', IntegerType(), True), StructField('fe_4', IntegerType(), True), StructField('fe_5', IntegerType(), True), StructField('fe_6', IntegerType(), True), StructField('fe_7', IntegerType(), True), StructField('fe_8', IntegerType(), True), StructField('fe_9', IntegerType(), True), StructField('fe_10', IntegerType(), True), StructField('fe_11', IntegerType(), True), StructField('fe_12', IntegerType(), True), StructField('fe_13', IntegerType(), True), StructField('fe_14', IntegerType(), True), StructField('fe_15', IntegerType(), True), StructField('fe_16', IntegerType(), True), StructField('fe_17', IntegerType(), True), StructField('fe_18', IntegerType(), True), StructFi

In [7]:
bronze_directory = 'datamart/bronze/features_financials'
silver_directory = 'datamart/silver/features_financials_cleaned'
processor.process_bronze_to_silver(
    bronze_path=bronze_directory,
    silver_path=silver_directory,
    dates=dates_str_lst,
    source_type="features_financials"
)

2025-05-21 04:37:55,068 - INFO - Schema validation passed.
2025-05-21 04:37:55,411 - INFO - Processed DataFrame count for 2023-01-01: 278
2025-05-21 04:37:55,413 - INFO - Processed DataFrame schema for 2023-01-01: StructType([StructField('Customer_ID', StringType(), True), StructField('Annual_Income', FloatType(), True), StructField('Monthly_Inhand_Salary', FloatType(), True), StructField('Num_Bank_Accounts', IntegerType(), True), StructField('Num_Credit_Card', IntegerType(), True), StructField('Interest_Rate', IntegerType(), True), StructField('Num_of_Loan', IntegerType(), True), StructField('Delay_from_due_date', IntegerType(), True), StructField('Num_of_Delayed_Payment', FloatType(), True), StructField('Changed_Credit_Limit', FloatType(), True), StructField('Num_Credit_Inquiries', IntegerType(), True), StructField('Credit_Mix', IntegerType(), True), StructField('Outstanding_Debt', FloatType(), True), StructField('Credit_Utilization_Ratio', FloatType(), True), StructField('Payment_of

In [8]:
bronze_directory = 'datamart/bronze/lms_loan_daily'
silver_directory = 'datamart/silver/lms_loan_daily_cleaned'
processor.process_bronze_to_silver(
    bronze_path=bronze_directory,
    silver_path=silver_directory,
    dates=dates_str_lst,
    source_type="lms_loan_daily"
)

2025-05-21 04:38:35,386 - INFO - Schema validation passed.
2025-05-21 04:38:35,479 - INFO - Processed DataFrame count for 2023-01-01: 530
2025-05-21 04:38:35,481 - INFO - Processed DataFrame schema for 2023-01-01: StructType([StructField('loan_id', StringType(), True), StructField('Customer_ID', StringType(), True), StructField('loan_start_date', DateType(), True), StructField('tenure', IntegerType(), True), StructField('installment_num', IntegerType(), True), StructField('loan_amt', FloatType(), True), StructField('due_amt', FloatType(), True), StructField('paid_amt', FloatType(), True), StructField('overdue_amt', FloatType(), True), StructField('balance', FloatType(), True), StructField('_metadata_file_name', StringType(), True), StructField('snapshot_date', DateType(), True)])
2025-05-21 04:38:35,481 - INFO - Processed schema validation passed.
2025-05-21 04:38:35,620 - INFO - Data successfully appended to datamart/silver/lms_loan_daily_cleaned.
2025-05-21 04:38:35,621 - INFO - Succ

In [9]:
bronze_directory = 'datamart/bronze/features_attributes'
silver_directory = 'datamart/silver/features_attributes_cleaned'
processor.process_bronze_to_silver(
    bronze_path=bronze_directory,
    silver_path=silver_directory,
    dates=dates_str_lst,
    source_type="features_attributes"
)

2025-05-21 04:38:45,752 - INFO - Schema validation passed.
2025-05-21 04:38:45,828 - INFO - Processed DataFrame count for 2023-01-01: 495
2025-05-21 04:38:45,828 - INFO - Processed DataFrame schema for 2023-01-01: StructType([StructField('Customer_ID', StringType(), True), StructField('Name', StringType(), True), StructField('Age', IntegerType(), True), StructField('SSN', StringType(), True), StructField('Occupation', StringType(), True), StructField('_metadata_file_name', StringType(), True), StructField('snapshot_date', DateType(), True)])
2025-05-21 04:38:45,829 - INFO - Processed schema validation passed.
2025-05-21 04:38:45,928 - INFO - Data successfully appended to datamart/silver/features_attributes_cleaned.
2025-05-21 04:38:45,929 - INFO - Successfully processed features_attributes for 2023-01-01
2025-05-21 04:38:45,958 - INFO - Schema validation passed.
2025-05-21 04:38:46,005 - INFO - Processed DataFrame count for 2023-02-01: 472
2025-05-21 04:38:46,006 - INFO - Processed Dat

In [10]:
silver_builder = utils.data_processing_silver_table.SilverDataMart(spark)
silver_builder.run(write_mode="overwrite", preview=True)

2025-05-21 04:38:52,164 - INFO - Loading cleaned silver inputs …
2025-05-21 04:38:52,386 - INFO - Input row counts – LMS: 104,288, Clickstream: 215,376, Attr: 11,298, Fin: 6,576
2025-05-21 04:38:52,583 - INFO - Writing dim_customer  to  datamart/silver/dim_customer
2025-05-21 04:38:52,850 - INFO - Writing dim_feature  to  datamart/silver/dim_feature
2025-05-21 04:38:53,375 - INFO - Writing dim_credit_mix  to  datamart/silver/dim_credit_mix
2025-05-21 04:38:53,478 - INFO - Writing dim_payment_behaviour  to  datamart/silver/dim_payment_behaviour
2025-05-21 04:38:53,594 - INFO - Writing dim_min_payment  to  datamart/silver/dim_min_payment
2025-05-21 04:38:53,695 - INFO - Writing dim_loan_type  to  datamart/silver/dim_loan_type
2025-05-21 04:38:53,869 - INFO - Writing fact_clickstream  to  datamart/silver/fact_clickstream
2025-05-21 04:38:55,402 - INFO - Writing fact_financials  to  datamart/silver/fact_financials
2025-05-21 04:38:55,514 - INFO - Writing fact_loan  to  datamart/silver/fact

root
 |-- customer_id: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Occupation: string (nullable = true)

+-----------+-----------+---------+---+-------------+
|customer_id|SSN        |Name     |Age|Occupation   |
+-----------+-----------+---------+---+-------------+
|cus_0x6a1d |455-79-2070|Suzannex |44 |Media_Manager|
|cus_0x7ff3 |300-00-5065|Oliviau  |45 |Developer    |
|cus_0x4854 |564-50-3217|Langep   |44 |Accountant   |
|cus_0xb0bd |744-86-5309|Herberta |37 |Accountant   |
|cus_0x4fe9 |296-37-3283|Schladerm|23 |Doctor       |
+-----------+-----------+---------+---+-------------+
only showing top 5 rows

root
 |-- feature_id: long (nullable = true)
 |-- feature_name: string (nullable = true)



2025-05-21 04:38:56,256 - INFO - 
Schema – dim_credit_mix
2025-05-21 04:38:56,312 - INFO - 
Schema – dim_payment_behaviour
2025-05-21 04:38:56,372 - INFO - 
Schema – dim_min_payment
2025-05-21 04:38:56,428 - INFO - 
Schema – dim_loan_type


+----------+------------+
|feature_id|feature_name|
+----------+------------+
|1         |fe_1        |
|2         |fe_2        |
|3         |fe_3        |
|4         |fe_4        |
|5         |fe_5        |
+----------+------------+
only showing top 5 rows

root
 |-- credit_mix_id: integer (nullable = true)

+-------------+
|credit_mix_id|
+-------------+
|1            |
|2            |
|0            |
+-------------+

root
 |-- payment_behaviour_id: string (nullable = true)
 |-- spend: string (nullable = true)
 |-- value: string (nullable = true)

+--------------------------------+-----+------+
|payment_behaviour_id            |spend|value |
+--------------------------------+-----+------+
|Low_spent_Medium_value_payments |Low  |Medium|
|Low_spent_Large_value_payments  |Low  |Large |
|High_spent_Medium_value_payments|High |Medium|
|High_spent_Small_value_payments |High |Small |
|Low_spent_Small_value_payments  |Low  |Small |
+--------------------------------+-----+------+
only showing

2025-05-21 04:38:56,585 - INFO - 
Schema – fact_clickstream
2025-05-21 04:38:56,721 - INFO - 
Schema – fact_financials
2025-05-21 04:38:56,755 - INFO - 
Schema – fact_loan


+------------+-----------------------+-------------------+
|loan_type_id|loan_flag_col          |loan_type_name     |
+------------+-----------------------+-------------------+
|1           |Has_Credit-Builder_Loan|Credit-Builder_Loan|
|2           |Has_Student_Loan       |Student_Loan       |
|3           |Has_Mortgage_Loan      |Mortgage_Loan      |
|4           |Has_Payday_Loan        |Payday_Loan        |
|5           |Has_Personal_Loan      |Personal_Loan      |
+------------+-----------------------+-------------------+
only showing top 5 rows

root
 |-- customer_id: string (nullable = true)
 |-- feature_id: long (nullable = true)
 |-- value: integer (nullable = true)
 |-- snapshot_date: date (nullable = true)

+-----------+----------+-----+-------------+
|customer_id|feature_id|value|snapshot_date|
+-----------+----------+-----+-------------+
|CUS_0x1037 |2         |243  |2024-03-01   |
|CUS_0x1037 |6         |56   |2024-03-01   |
|CUS_0x1037 |3         |89   |2024-03-01   |
|CUS

2025-05-21 04:38:56,789 - INFO - 
Schema – fact_customer_loan_type


+---------------------+-----------+---+-------------------+-----------------+---+-------------+---------------+------+--------+-------+--------+-----------+-------+
|loan_id              |customer_id|mob|installments_missed|first_missed_date|dpd|snapshot_date|loan_start_date|tenure|loan_amt|due_amt|paid_amt|overdue_amt|balance|
+---------------------+-----------+---+-------------------+-----------------+---+-------------+---------------+------+--------+-------+--------+-----------+-------+
|CUS_0x5012_2023_11_01|cus_0x5012 |9  |5                  |2024-03-01       |153|2024-08-01   |2023-11-01     |10    |10000.0 |1000.0 |0.0     |5000.0     |6000.0 |
|CUS_0x501c_2024_03_01|cus_0x501c |5  |2                  |2024-06-01       |61 |2024-08-01   |2024-03-01     |10    |10000.0 |1000.0 |0.0     |2000.0     |7000.0 |
|CUS_0x501d_2024_02_01|cus_0x501d |6  |0                  |NULL             |0  |2024-08-01   |2024-02-01     |10    |10000.0 |1000.0 |1000.0  |0.0        |4000.0 |
|CUS_0x503

## Build gold table for features and labels

In [11]:
proc = utils.data_processing_gold_table.GoldDataProcessor(spark)
proc.build_loan_analytics_star()
proc.build_label_store(dpd_threshold=30, mob_threshold=6)

2025-05-21 04:38:56,949 - INFO - Reading Silver table 'dim_credit_mix' from datamart/silver/dim_credit_mix
2025-05-21 04:38:56,973 - INFO - Reading Silver table 'dim_payment_behaviour' from datamart/silver/dim_payment_behaviour
2025-05-21 04:38:56,996 - INFO - Reading Silver table 'dim_min_payment' from datamart/silver/dim_min_payment
2025-05-21 04:38:57,018 - INFO - Reading Silver table 'dim_loan_type' from datamart/silver/dim_loan_type
2025-05-21 04:38:57,040 - INFO - Reading Silver table 'dim_customer' from datamart/silver/dim_customer
2025-05-21 04:38:57,068 - INFO - Reading Silver table 'fact_loan' from datamart/silver/fact_loan
2025-05-21 04:38:57,094 - INFO - Reading Silver table 'fact_financials' from datamart/silver/fact_financials
2025-05-21 04:38:57,152 - INFO - Reading Silver table 'fact_customer_loan_type' from datamart/silver/fact_customer_loan_type
2025-05-21 04:39:00,841 - INFO - Writing Gold table 'loan_analytics' to datamart/gold/loan_analytics_star, partitioned by ['

+-----------+------------+---+------------------------+--------------------+---------------------+-----------------+--------------------+-----------------------+---------------------------+------------------------+-------------------+--------------------+--------------------+----------------------+----------------------------+-----------+-------------+
|customer_id|        Name|Age|Credit_Utilization_Ratio|Credit_History_Years|Credit_History_Months|Annual_Income_Log|Outstanding_Debt_Log|Total_EMI_per_month_Log|Amount_invested_monthly_Log|Num_Credit_Inquiries_Log|Monthly_Balance_Log|Debt_to_Income_Ratio| EMI_to_Income_Ratio|Invest_to_Income_Ratio|Credit_Inquiry_to_Loan_Ratio|Is_Employed|snapshot_date|
+-----------+------------+---+------------------------+--------------------+---------------------+-----------------+--------------------+-----------------------+---------------------------+------------------------+-------------------+--------------------+--------------------+--------------

2025-05-21 04:39:01,251 - INFO - Reading silver fact_loan for all snapshot_dates
2025-05-21 04:39:01,251 - INFO - Reading Silver table 'fact_loan' from datamart/silver/fact_loan
2025-05-21 04:39:01,281 - INFO - Writing label_store for all dates (30dpd/6mob)
2025-05-21 04:39:01,283 - INFO - Writing Gold table 'label_store' to datamart/gold/label_store, partitioned by ['snapshot_date']


DataFrame[loan_id: string, customer_id: string, label: int, label_def: string, snapshot_date: date]